In [35]:
# Manipulação de dados
import pandas as pd

# Utilidades
from tqdm import tqdm

# Configuração de sistema
import sys, os

# Módulos autorais
sys.path.append('../')
from config.config import *
from modulos.check_sanidade import mostrar_visao_geral, testa_granularidade, tb_ausentes, tb_distintos, obter_duplicatas, criar_chv_cols, tb_distrib, tb_freq

In [5]:
def le_csv(pasta_arquivo, nm_arquivo):

    caminho_arquivo = os.path.join(pasta_arquivo, nm_arquivo)

    try:
        df = pd.read_csv(caminho_arquivo, sep = ',', encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(caminho_arquivo, sep = ',', encoding='latin1')
    finally:
        df['Tabela'] = nm_arquivo
    
    return df

def le_pastas_csv(lst_pastas):

    lst_dfs = []

    for pasta in lst_pastas:

        for nm_arquivo in os.listdir(pasta):
            if nm_arquivo.endswith('.csv'):
                # print(nm_arquivo)
                
                df = le_csv(pasta, nm_arquivo)
                lst_dfs.append(df)

    return pd.concat(lst_dfs)

In [193]:
ar_sp = le_pastas_csv([PASTA_DADOS_QUALIDADE_AR_SP])

# Visão geral

In [7]:
mostrar_visao_geral(ar_sp)

Tamanho da base: (12663857, 9)

Descrição das colunas:

<class 'pandas.core.frame.DataFrame'>
Index: 12663857 entries, 0 to 808417
Data columns (total 9 columns):
 #   Column    Dtype  
---  ------    -----  
 0   Data      object 
 1   Hora      object 
 2   Estacao   object 
 3   Codigo    object 
 4   Poluente  object 
 5   Valor     float64
 6   Unidade   object 
 7   Tipo      object 
 8   Tabela    object 
dtypes: float64(1), object(8)
memory usage: 966.2+ MB
None

Visualização dos primeiros registros:


,Data,Hora,Estacao,Codigo,Poluente,Valor,Unidade,Tipo,Tabela
0,2015-01-01,01:00,Americana - Vila Santa Maria,SP01,MP10,65.0,ug/m3,automatica,SP201501.csv
1,2015-01-01,02:00,Americana - Vila Santa Maria,SP01,MP10,98.0,ug/m3,automatica,SP201501.csv
2,2015-01-01,03:00,Americana - Vila Santa Maria,SP01,MP10,79.0,ug/m3,automatica,SP201501.csv
3,2015-01-01,04:00,Americana - Vila Santa Maria,SP01,MP10,53.0,ug/m3,automatica,SP201501.csv
4,2015-01-01,05:00,Americana - Vila Santa Maria,SP01,MP10,35.0,ug/m3,automatica,SP201501.csv


# Granularidade

In [26]:
chv = ['Estacao','Data','Hora','Poluente']

In [48]:
testa_granularidade(ar_sp, chv)

Qtd de linhas da base:
12663857
Qtd de combinacoes distintas:
12663760


In [19]:
dupl = obter_duplicatas(ar_sp, chv)
dupl

,Data,Hora,Estacao,Codigo,Poluente,Valor,Unidade,Tipo,Tabela,chv,qtd_distintos_chv
224501,2019-01-01,01:00,Cerqueira Cesar,SP14,SO2,3.0,ug/m3,automatica,SP201901.csv,Cerqueira Cesar | 2019-01-01 | 01:00 | SO2,2
803525,2019-01-01,01:00,Cerqueira Cesar,SP14,SO2,3.0,ug/m3,manual,SP201902.csv,Cerqueira Cesar | 2019-01-01 | 01:00 | SO2,2
225076,2019-02-01,01:00,Cerqueira Cesar,SP14,SO2,24.0,ug/m3,automatica,SP201901.csv,Cerqueira Cesar | 2019-02-01 | 01:00 | SO2,2
803526,2019-02-01,01:00,Cerqueira Cesar,SP14,SO2,3.0,ug/m3,manual,SP201902.csv,Cerqueira Cesar | 2019-02-01 | 01:00 | SO2,2
226041,2019-04-01,01:00,Cerqueira Cesar,SP14,SO2,5.0,ug/m3,automatica,SP201901.csv,Cerqueira Cesar | 2019-04-01 | 01:00 | SO2,2
...,...,...,...,...,...,...,...,...,...,...,...
814655,2018-12-26,01:00,Santa Gertrudes,SP57,MP2.5,16.0,ug/m3,manual,SP201802.csv,Santa Gertrudes | 2018-12-26 | 01:00 | MP2.5,2
760356,2021-07-11,01:00,Santo Andre - Capuava,SP61,MP2.5,39.0,ug/m3,manual,SP202102.csv,Santo Andre - Capuava | 2021-07-11 | 01:00 | M...,2
760357,2021-07-11,01:00,Santo Andre - Capuava,SP61,MP2.5,39.0,ug/m3,manual,SP202102.csv,Santo Andre - Capuava | 2021-07-11 | 01:00 | M...,2
740613,2016-04-05,01:00,Sorocaba - Centro,SP74,FMC,15.0,ug/m3,manual,SP201602.csv,Sorocaba - Centro | 2016-04-05 | 01:00 | FMC,2


In [21]:
(dupl
    .groupby(chv)
    .agg({'Tipo':pd.Series.nunique})
    .reset_index())

,Estacao,Data,Hora,Poluente,Tipo
0,Cerqueira Cesar,2019-01-01,01:00,SO2,2
1,Cerqueira Cesar,2019-02-01,01:00,SO2,2
2,Cerqueira Cesar,2019-04-01,01:00,SO2,2
3,Cerqueira Cesar,2019-05-01,01:00,SO2,2
4,Cerqueira Cesar,2019-06-01,01:00,SO2,2
...,...,...,...,...,...
92,Santa Gertrudes,2018-12-14,01:00,MP2.5,2
93,Santa Gertrudes,2018-12-20,01:00,MP2.5,2
94,Santa Gertrudes,2018-12-26,01:00,MP2.5,2
95,Santo Andre - Capuava,2021-07-11,01:00,MP2.5,1


In [23]:
(dupl
    .groupby(chv)
    .agg({'Tipo':pd.Series.nunique})
    .reset_index()
    .iloc[-2:])

,Estacao,Data,Hora,Poluente,Tipo
95,Santo Andre - Capuava,2021-07-11,01:00,MP2.5,1
96,Sorocaba - Centro,2016-04-05,01:00,FMC,1


In [27]:
(dupl
    .groupby(chv)
    .agg({'Tipo':pd.Series.nunique})
    .reset_index()
    .iloc[-2:]
    .merge(
        dupl, how = 'left', on=chv
    ))

,Estacao,Data,Hora,Poluente,Tipo_x,Codigo,Valor,Unidade,Tipo_y,Tabela,chv,qtd_distintos_chv
0,Santo Andre - Capuava,2021-07-11,01:00,MP2.5,1,SP61,39.0,ug/m3,manual,SP202102.csv,Santo Andre - Capuava | 2021-07-11 | 01:00 | M...,2
1,Santo Andre - Capuava,2021-07-11,01:00,MP2.5,1,SP61,39.0,ug/m3,manual,SP202102.csv,Santo Andre - Capuava | 2021-07-11 | 01:00 | M...,2
2,Sorocaba - Centro,2016-04-05,01:00,FMC,1,SP74,15.0,ug/m3,manual,SP201602.csv,Sorocaba - Centro | 2016-04-05 | 01:00 | FMC,2
3,Sorocaba - Centro,2016-04-05,01:00,FMC,1,SP74,15.0,ug/m3,manual,SP201602.csv,Sorocaba - Centro | 2016-04-05 | 01:00 | FMC,2


Para os casos em que há registros com mais de uma medição na mesma chave [Estação + Data + Hora + Poluente], serão consideradas as medições manuais.

In [29]:
import numpy as np

In [33]:
ar_sp.shape[0]

12663857

In [34]:
ar_sp['possui_duplicata'] = ar_sp.duplicated(subset=chv, keep=False)

In [36]:
tb_freq(ar_sp,'possui_duplicata')

,possui_duplicata,count,proportion
0,False,12663663,0.999985
1,True,194,0.000015


In [38]:
ar_sp[
    (ar_sp.possui_duplicata) &
    (ar_sp['Tipo']=='manual')]

,Data,Hora,Estacao,Codigo,Poluente,Valor,Unidade,Tipo,Tabela,chv,qtd_distintos_chv,possui_duplicata
687778,2015-01-01,01:00,Cerqueira César,SP14,SO2,3.0,ug/m3,manual,SP201502.csv,Cerqueira César | 2015-01-01 | 01:00 | SO2,2,True
687779,2015-02-01,01:00,Cerqueira César,SP14,SO2,3.0,ug/m3,manual,SP201502.csv,Cerqueira César | 2015-02-01 | 01:00 | SO2,2,True
687780,2015-03-01,01:00,Cerqueira César,SP14,SO2,3.0,ug/m3,manual,SP201502.csv,Cerqueira César | 2015-03-01 | 01:00 | SO2,2,True
687781,2015-04-01,01:00,Cerqueira César,SP14,SO2,3.0,ug/m3,manual,SP201502.csv,Cerqueira César | 2015-04-01 | 01:00 | SO2,2,True
687782,2015-05-01,01:00,Cerqueira César,SP14,SO2,3.0,ug/m3,manual,SP201502.csv,Cerqueira César | 2015-05-01 | 01:00 | SO2,2,True
...,...,...,...,...,...,...,...,...,...,...,...,...
808396,2022-05-01,01:00,Pinheiros,SP50,SO2,3.0,ug/m3,manual,SP202202.csv,Pinheiros | 2022-05-01 | 01:00 | SO2,2,True
808397,2022-06-01,01:00,Pinheiros,SP50,SO2,3.0,ug/m3,manual,SP202202.csv,Pinheiros | 2022-06-01 | 01:00 | SO2,2,True
808398,2022-07-01,01:00,Pinheiros,SP50,SO2,3.0,ug/m3,manual,SP202202.csv,Pinheiros | 2022-07-01 | 01:00 | SO2,2,True
808399,2022-08-01,01:00,Pinheiros,SP50,SO2,3.0,ug/m3,manual,SP202202.csv,Pinheiros | 2022-08-01 | 01:00 | SO2,2,True


In [51]:
ar_sp[
    (ar_sp.possui_duplicata) &
    (ar_sp['Tipo']=='automatica')]

,Data,Hora,Estacao,Codigo,Poluente,Valor,Unidade,Tipo,Tabela,chv,qtd_distintos_chv,possui_duplicata
236589,2015-01-01,01:00,Cerqueira César,SP14,SO2,2.0,ug/m3,automatica,SP201501.csv,Cerqueira César | 2015-01-01 | 01:00 | SO2,2,True
237283,2015-02-01,01:00,Cerqueira César,SP14,SO2,0.0,ug/m3,automatica,SP201501.csv,Cerqueira César | 2015-02-01 | 01:00 | SO2,2,True
237886,2015-03-01,01:00,Cerqueira César,SP14,SO2,1.0,ug/m3,automatica,SP201501.csv,Cerqueira César | 2015-03-01 | 01:00 | SO2,2,True
238591,2015-04-01,01:00,Cerqueira César,SP14,SO2,2.0,ug/m3,automatica,SP201501.csv,Cerqueira César | 2015-04-01 | 01:00 | SO2,2,True
239183,2015-05-01,01:00,Cerqueira César,SP14,SO2,1.0,ug/m3,automatica,SP201501.csv,Cerqueira César | 2015-05-01 | 01:00 | SO2,2,True
...,...,...,...,...,...,...,...,...,...,...,...,...
220595,2022-04-01,01:00,Cerqueira Cesar,SP14,SO2,0.0,ug/m3,automatica,SP202201.csv,Cerqueira Cesar | 2022-04-01 | 01:00 | SO2,2,True
221748,2022-06-01,01:00,Cerqueira Cesar,SP14,SO2,1.0,ug/m3,automatica,SP202201.csv,Cerqueira Cesar | 2022-06-01 | 01:00 | SO2,2,True
222420,2022-07-01,01:00,Cerqueira Cesar,SP14,SO2,3.0,ug/m3,automatica,SP202201.csv,Cerqueira Cesar | 2022-07-01 | 01:00 | SO2,2,True
223115,2022-08-01,01:00,Cerqueira Cesar,SP14,SO2,3.0,ug/m3,automatica,SP202201.csv,Cerqueira Cesar | 2022-08-01 | 01:00 | SO2,2,True


In [55]:
ar_sp[
    (ar_sp.possui_duplicata) &
    (ar_sp['Tipo']=='automatica')]

,Data,Hora,Estacao,Codigo,Poluente,Valor,Unidade,Tipo,Tabela,chv,qtd_distintos_chv,possui_duplicata
236589,2015-01-01,01:00,Cerqueira César,SP14,SO2,2.0,ug/m3,automatica,SP201501.csv,Cerqueira César | 2015-01-01 | 01:00 | SO2,2,True
237283,2015-02-01,01:00,Cerqueira César,SP14,SO2,0.0,ug/m3,automatica,SP201501.csv,Cerqueira César | 2015-02-01 | 01:00 | SO2,2,True
237886,2015-03-01,01:00,Cerqueira César,SP14,SO2,1.0,ug/m3,automatica,SP201501.csv,Cerqueira César | 2015-03-01 | 01:00 | SO2,2,True
238591,2015-04-01,01:00,Cerqueira César,SP14,SO2,2.0,ug/m3,automatica,SP201501.csv,Cerqueira César | 2015-04-01 | 01:00 | SO2,2,True
239183,2015-05-01,01:00,Cerqueira César,SP14,SO2,1.0,ug/m3,automatica,SP201501.csv,Cerqueira César | 2015-05-01 | 01:00 | SO2,2,True
...,...,...,...,...,...,...,...,...,...,...,...,...
220595,2022-04-01,01:00,Cerqueira Cesar,SP14,SO2,0.0,ug/m3,automatica,SP202201.csv,Cerqueira Cesar | 2022-04-01 | 01:00 | SO2,2,True
221748,2022-06-01,01:00,Cerqueira Cesar,SP14,SO2,1.0,ug/m3,automatica,SP202201.csv,Cerqueira Cesar | 2022-06-01 | 01:00 | SO2,2,True
222420,2022-07-01,01:00,Cerqueira Cesar,SP14,SO2,3.0,ug/m3,automatica,SP202201.csv,Cerqueira Cesar | 2022-07-01 | 01:00 | SO2,2,True
223115,2022-08-01,01:00,Cerqueira Cesar,SP14,SO2,3.0,ug/m3,automatica,SP202201.csv,Cerqueira Cesar | 2022-08-01 | 01:00 | SO2,2,True


In [56]:
ar_sp = ar_sp.drop(
    ar_sp[
        (ar_sp.possui_duplicata) &
        (ar_sp['Tipo']=='automatica')
    ].index
)

In [59]:
ar_sp = ar_sp.drop_duplicates()

In [60]:
testa_granularidade(ar_sp, chv)

Qtd de linhas da base:
12662515
Qtd de combinacoes distintas:
12662515


# Valores ausentes

In [66]:
ausentes = tb_ausentes(ar_sp)
ausentes

Há 12 colunas com valores ausentes.


,qtd_ausentes,pct_ausentes
Codigo,31397,0.25
Data,0,0.00
Hora,0,0.00
Estacao,0,0.00
Poluente,0,0.00
Valor,0,0.00
Unidade,0,0.00
Tipo,0,0.00
Tabela,0,0.00
chv,0,0.00


Foram identificados valores ausentes na coluna:
- Codigo

# Valores distintos

In [67]:
distintos = tb_distintos(ar_sp)
distintos

,col,qtd_distintos,pct_distintos
9,chv,12662515,100.00
0,Data,2922,0.02
5,Valor,752,0.01
1,Hora,25,0.00
2,Estacao,129,0.00
3,Codigo,88,0.00
4,Poluente,9,0.00
6,Unidade,1,0.00
7,Tipo,2,0.00
8,Tabela,16,0.00


# Categorização das colunas

In [ ]:
cols_cat = ['Estacao','Codigo','Poluente','Unidade','Tipo']

cols_dat = ['Data', 'Hora']

cols_num = ['Valor']

novas_cols = []

# Análise da colunas categóricas

In [65]:
cols_cat

['Estacao', 'Codigo', 'Poluente', 'Unidade', 'Tipo']

## Estacao

In [144]:
col = 'Estacao'
col_plural = 'Estações'

In [71]:
ausentes[ausentes.index==col]

,qtd_ausentes,pct_ausentes
Estacao,0,0.0


In [73]:
distintos[distintos.col==col]

,col,qtd_distintos,pct_distintos
2,Estacao,129,0.0


In [124]:
tb_freq(ar_sp, [col])

,Estacao,freq_abs,freq_rel
0,Santos - Ponta da Praia,328962,2.60
1,Parque Dom Pedro II,327687,2.59
2,Guarulhos - Pimentas,324866,2.57
3,Congonhas,320568,2.53
4,Osasco,310691,2.45
...,...,...,...
124,Paulínia - Bairro Cascata,40,0.00
125,Ribeirão Preto - Campos Elíseos,27,0.00
126,Paulinia - Bairro Cascata,26,0.00
127,Paulinia - Joao Aranha,26,0.00


In [160]:
import unidecode
import re

In [161]:
def padronizar_string(s):
    s = unidecode(s)
    s = s.upper()
    s = ' '.join(s.split())
    s = s.strip()
    s = re.sub('[^a-zA-Z0-9 \n\.]', '', s)
    return s

In [165]:
ar_sp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12662515 entries, 0 to 808417
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Data               object 
 1   Hora               object 
 2   Estacao            object 
 3   Codigo             object 
 4   Poluente           object 
 5   Valor              float64
 6   Unidade            object 
 7   Tipo               object 
 8   Tabela             object 
 9   chv                object 
 10  qtd_distintos_chv  int64  
 11  possui_duplicata   bool   
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 1.1+ GB


In [174]:
ar_sp['Data'].max()

'2022-12-31'

In [196]:
def converter_cols_data(df, nm_cols_data, formato='%Y-%M-%d'):
    new_df = df.copy()
    for col in nm_cols_data:
        new_df[col] = pd.to_datetime(new_df[col], format = formato)
    return new_df

TypeError: 'numpy.dtypes.ObjectDType' object is not callable

In [197]:
(ar_sp
    .assign(data_hora = lambda df: df['Data'] + ' ' + df['Hora'])
    .pipe(converter_cols_data,['data_hora'], formato = '%Y-%M-%d HH:mm')
    # .info()
)

ValueError: time data "2015-01-01 01:00" doesn't match format "%Y-%M-%d HH:mm", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [189]:
pd.to_datetime(ar_sp['Data'], format='%Y-%M-%d')

0        2015-01-01
1        2015-01-01
2        2015-01-01
3        2015-01-01
4        2015-01-01
            ...    
808413   2022-01-01
808414   2022-01-01
808415   2022-01-01
808416   2022-01-01
808417   2022-01-01
Name: Data, Length: 12662515, dtype: datetime64[ns]

In [163]:
ar_sp['Estacao'].apply(lambda x: padronizar_string(x))

TypeError: 'module' object is not callable

In [154]:
nome_estacao = ar_sp['Estacao'].to_numpy()
nome_estacao

array(['Americana - Vila Santa Maria', 'Americana - Vila Santa Maria',
       'Americana - Vila Santa Maria', ..., 'Tatuape', 'Tatuape',
       'Tatuape'], dtype=object)

In [155]:
from sklearn.metrics.pairwise import cosine_similarity

def similaridade_cos(array1, array2):
    return cosine_similarity(array1.reshape(1,-1), array2.reshape(1,-1))

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer


# Exemplo de DataFrame
data = {'texto1': ['Esta é uma frase de exemplo', 'Isso é um teste', 'Python é incrível'],
        'texto2': ['Uma frase de teste', 'Exemplo de código Python', 'Python é legal']}
df = pd.DataFrame(data)

# Inicializa o vetorizador CountVectorizer
vectorizer = CountVectorizer()

# Transforma as colunas de texto em matrizes de contagem de palavras
matrix_texto1 = vectorizer.fit_transform(df['texto1'])
matrix_texto2 = vectorizer.fit_transform(df['texto2'])

# Calcula a similaridade de cosseno entre as colunas de texto
similaridade = cosine_similarity(matrix_texto1, matrix_texto2)

# Cria um novo DataFrame com os valores de similaridade
df_similaridade = pd.DataFrame(similaridade, columns=['Similaridade'])

# Exibe o DataFrame de similaridade
print(df_similaridade)


In [145]:
ar_sp[col].sort_values().unique()

array(['Americana', 'Americana - Vila Santa Maria', 'Aracatuba',
       'Araraquara', 'Araçatuba', 'Barretos - América', 'Bauru',
       'Campinas - Centro', 'Campinas - Taquaral',
       'Campinas - Vila Uniao', 'Campinas - Vila União', 'Campos Eliseos',
       'Campos Elíseos', 'Capao Redondo', 'Capão Redondo', 'Carapicuiba',
       'Carapicuíba', 'Catanduva', 'Cerqueira Cesar', 'Cerqueira César',
       'Cid.Universitaria USP - IPEN', 'Cid.Universitária USP - IPEN',
       'Congonhas', 'Cordeiropolis - Modolo', 'Cordeirópolis - Módolo',
       'Cubatao - Centro', 'Cubatao - Vale do Mogi',
       'Cubatao - Vila Parisi', 'Cubatão - Centro',
       'Cubatão - Vale do Mogi', 'Cubatão - Vila Parisi', 'Diadema',
       'Franca - Centro', 'Franca - Cidade Nova', 'Grajau-Parelheiros',
       'Grajaú - Parelheiros', 'Guaratingueta', 'Guaratinguetá',
       'Guaruja - Vicente de Carvalho', 'Guarujá - Vicente de Carvalho',
       'Guarulhos - Paco Municipal', 'Guarulhos - Paço Municipal',
   

In [151]:
def extrair_municipio(str):
    arr_str = str.split('-')
    return [s.strip() for s in arr_str]

    
# # Remove espaços extras da coluna "coluna_texto"
# df['coluna_texto'] = df['coluna_texto'].str.strip()

In [152]:
str = 'Grajaú - Parelheiros'
extrair_municipio(str)

['Grajaú', 'Parelheiros']

In [ ]:
extrair_municipio(str)

In [140]:
path_dicionario_municipio = os.path.join(PASTA_DADOS_ENTRADA,'dicionario_cod_municipio.csv')

municipio = pd.read_csv(path_dicionario_municipio)

In [142]:
municipio[municipio['sigla_uf']=='SP']

,nome_municipio,cod_municipio,sigla_uf
1,Adamantina,3500105,SP
2,Adolfo,3500204,SP
3,Aguaí,3500303,SP
4,Águas da Prata,3500402,SP
5,Águas de Lindóia,3500501,SP
...,...,...,...
641,Vista Alegre do Alto,3556909,SP
642,Vitória Brasil,3556958,SP
643,Votorantim,3557006,SP
644,Votuporanga,3557105,SP


## Codigo

In [119]:
col = 'Codigo'
col_plural = 'Códigos'

In [120]:
ausentes[ausentes.index==col]

,qtd_ausentes,pct_ausentes
Codigo,31397,0.25


In [121]:
distintos[distintos.col==col]

,col,qtd_distintos,pct_distintos
3,Codigo,88,0.0


In [122]:
tb_freq(ar_sp, [col])

,Codigo,freq_abs,freq_rel
0,SP67,361465,2.86
1,SP64,328962,2.60
2,SP44,327687,2.59
3,SP37,325211,2.57
4,SP25,324866,2.57
...,...,...,...
83,SP132,89,0.00
84,SP05,74,0.00
85,SP47,67,0.00
86,SP46,66,0.00


## Poluente

In [125]:
col = 'Poluente'
col_plural = 'Poluentes'

In [126]:
ausentes[ausentes.index==col]

,qtd_ausentes,pct_ausentes
Poluente,0,0.0


In [127]:
distintos[distintos.col==col]

,col,qtd_distintos,pct_distintos
4,Poluente,9,0.0


In [128]:
tb_freq(ar_sp, [col])

,Poluente,freq_abs,freq_rel
0,MP10,3324914,26.26
1,O3,3078948,24.32
2,NO2,2469340,19.50
3,MP2.5,1483536,11.72
4,CO,1043549,8.24
5,SO2,908273,7.17
6,NO,347797,2.75
7,FMC,4127,0.03
8,PTS,2031,0.02


## Unidade

In [129]:
col = 'Unidade'
col_plural = 'Unidades'

In [130]:
ausentes[ausentes.index==col]

,qtd_ausentes,pct_ausentes
Unidade,0,0.0


In [131]:
distintos[distintos.col==col]

,col,qtd_distintos,pct_distintos
6,Unidade,1,0.0


In [132]:
tb_freq(ar_sp, [col])

,Unidade,freq_abs,freq_rel
0,ug/m3,12662515,100.0


## Codigo

In [133]:
col = 'Tipo'
col_plural = 'Tipos'

In [134]:
ausentes[ausentes.index==col]

,qtd_ausentes,pct_ausentes
Tipo,0,0.0


In [135]:
distintos[distintos.col==col]

,col,qtd_distintos,pct_distintos
7,Tipo,2,0.0


In [136]:
tb_freq(ar_sp, [col])

,Tipo,freq_abs,freq_rel
0,automatica,12652618,99.92
1,manual,9897,0.08


# Extração da base particionada por município

In [137]:
ar_sp

,Data,Hora,Estacao,Codigo,Poluente,Valor,Unidade,Tipo,Tabela,chv,qtd_distintos_chv,possui_duplicata
0,2015-01-01,01:00,Americana - Vila Santa Maria,SP01,MP10,65.0,ug/m3,automatica,SP201501.csv,Americana - Vila Santa Maria | 2015-01-01 | 01...,1,False
1,2015-01-01,02:00,Americana - Vila Santa Maria,SP01,MP10,98.0,ug/m3,automatica,SP201501.csv,Americana - Vila Santa Maria | 2015-01-01 | 02...,1,False
2,2015-01-01,03:00,Americana - Vila Santa Maria,SP01,MP10,79.0,ug/m3,automatica,SP201501.csv,Americana - Vila Santa Maria | 2015-01-01 | 03...,1,False
3,2015-01-01,04:00,Americana - Vila Santa Maria,SP01,MP10,53.0,ug/m3,automatica,SP201501.csv,Americana - Vila Santa Maria | 2015-01-01 | 04...,1,False
4,2015-01-01,05:00,Americana - Vila Santa Maria,SP01,MP10,35.0,ug/m3,automatica,SP201501.csv,Americana - Vila Santa Maria | 2015-01-01 | 05...,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
808413,2022-05-01,01:00,Tatuape,SP76,SO2,3.0,ug/m3,manual,SP202202.csv,Tatuape | 2022-05-01 | 01:00 | SO2,1,False
808414,2022-06-01,01:00,Tatuape,SP76,SO2,3.0,ug/m3,manual,SP202202.csv,Tatuape | 2022-06-01 | 01:00 | SO2,1,False
808415,2022-07-01,01:00,Tatuape,SP76,SO2,5.0,ug/m3,manual,SP202202.csv,Tatuape | 2022-07-01 | 01:00 | SO2,1,False
808416,2022-08-01,01:00,Tatuape,SP76,SO2,3.0,ug/m3,manual,SP202202.csv,Tatuape | 2022-08-01 | 01:00 | SO2,1,False
